In [ ]:
# @title تشغيل سكربت SiFThon - إصدار آمن (يدعم Colab + MyBinder)

import os
import requests
import zipfile
import subprocess
import sys
import socket

def verify_environment():
    print("جاري التحقق من البيئة...")

    checks = {
        'colab_environment': False,
        'google_domain': False,
        'binder_environment': False,
        'required_libraries': False
    }

    # تحقق من بيئة Google Colab
    if 'COLAB_GPU' in os.environ or 'COLAB_JUPYTER_TOKEN' in os.environ:
        checks['colab_environment'] = True

    # تحقق من اسم المضيف لتحديد Colab أو Binder
    try:
        hostname = socket.gethostname().lower()
        if 'colab' in hostname:
            checks['google_domain'] = True
        elif 'binder' in hostname or 'mybinder' in hostname:
            checks['binder_environment'] = True
    except Exception:
        pass

    # تحقق من وجود مكتبات Colab
    try:
        import google.colab  # noqa
        checks['required_libraries'] = True
    except ImportError:
        # Binder لا يملك google.colab
        checks['required_libraries'] = False

    print(f"نتائج التحقق: {checks}")

    # قبول البيئة إذا كانت Colab أو Binder
    if not (checks['colab_environment'] or checks['google_domain'] or checks['binder_environment']):
        print("❌ هذا السكربت يعمل فقط على Google Colab أو MyBinder")
        sys.exit(1)

    print("✅ تم التحقق بنجاح — بيئة مدعومة (Colab / MyBinder)")
    return True

def download_with_verification():
    url = 'http://especi1.serv00.net/url.php'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36',
        'Accept': 'application/zip, */*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,ar;q=0.8',
        'Referer': 'https://colab.research.google.com/',
        'Origin': 'https://colab.research.google.com',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'cross-site'
    }

    print("جاري الاتصال بالسيرفر...")
    try:
        response = requests.get(url, headers=headers, timeout=30, allow_redirects=True)
        print(f"كود الاستجابة: {response.status_code}")
        if response.status_code == 200:
            with open('SiFThon.zip', 'wb') as f:
                f.write(response.content)
            print("تم التحميل بنجاح")
            return True
        elif response.status_code == 403:
            print("الوصول مرفوض: التحقق من البيئة فشل")
            print("تأكد من أنك تستخدم Google Colab أو MyBinder")
            return False
        else:
            print(f"فشل التحميل: {response.status_code} - {response.reason}")
            return False
    except requests.exceptions.RequestException as e:
        print(f"خطأ في الاتصال: {e}")
        return False

def main():
    print("=" * 50)
    print("سكربت SiFThon - الإصدار الآمن (Colab + MyBinder)")
    print("=" * 50)

    if not verify_environment():
        return

    if not download_with_verification():
        print("فشل في تحميل السكربت")
        return

    try:
        print("جاري فك الضغط...")
        with zipfile.ZipFile('SiFThon.zip', 'r') as zip_ref:
            zip_ref.extractall('.')
        print("تم فك الضغط بنجاح")
    except zipfile.BadZipFile:
        print("خطأ: ملف ZIP تالف")
        return

    source_folder = None
    for root, dirs, files in os.walk('.'):
        if 'SiFThon' in dirs:
            source_folder = os.path.join(root, 'SiFThon')
            break

    if not source_folder:
        print("خطأ: لم يتم العثور على مجلد SiFThon")
        return

    os.chdir(os.path.dirname(source_folder))

    print("جاري تثبيت المكتبات المطلوبة...")
    requirements = [
        'telethon', 'pyrogram', 'requests', 'tabulate',
        'ipaddress', 'telegraph', 'python-decouple',
        'Pillow', 'pytz', 'gitpython', 'fpdf', 'youtubesearchpython', 'yt-dlp'
    ]

    for req in requirements:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', req], check=True)
            print(f"تم تثبيت: {req}")
        except subprocess.CalledProcessError:
            print(f"فشل تثبيت: {req}")

    print("\nأدخل بيانات الاتصال:")
    try:
        API_ID = input("API_ID: ")
        API_HASH = input("API_HASH: ")
        SESSION = input("SESSION: ")
        BOT_TOKEN = input("BOT_TOKEN: ")

        env_content = f"API_ID={API_ID}\nAPI_HASH={API_HASH}\nSESSION={SESSION}\nBOT_TOKEN={BOT_TOKEN}\n"
        with open('SiFThon/.env', 'w') as f:
            f.write(env_content)

        print("\nمحتويات ملف الإعدادات:")
        with open('SiFThon/.env', 'r') as file:
            print(file.read())

        print("جاري تشغيل السكربت...")
        print("=" * 40)
        process = subprocess.Popen(['python', '-m', 'SiFThon'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        try:
            for line in process.stdout:
                print(line, end='')
        except KeyboardInterrupt:
            process.terminate()
            print("\nتم إيقاف التشغيل يدوياً")
    except Exception as e:
        print(f"حدث خطأ: {e}")

if __name__ == '__main__':
    main()

---
## معلومات النظام
- اسم السكربت: SiFThon
- الإصدار: 1.1 الآمن (يدعم Colab + MyBinder)
- نوع الحماية: تحقق من Google Colab و MyBinder فقط
- المكتبات: Telethon, Pyrogram, إلخ

---
## متطلبات التشغيل
1. يجب التشغيل على Google Colab أو MyBinder فقط
2. اتصال انترنت مستقر
3. بيانات API صحيحة
4. جلسة تيليجرام سليمة

---
## ملاحظات هامة
- هذا السكربت محمي ضد التشغيل خارج Colab أو MyBinder
- يتم التحقق من البيئة تلقائياً
- الرفض يعني أنك لا تستخدم بيئة مدعومة
- لا تشارك بياناتك الشخصية

---
<h6 align='center'>© SiFThon - الإصدار الآمن 1.1</h6>